# Music Recommendation System using Collaborative Filtering with ListenBrainz

## Summary
---
This project aims to develop a music recommendation system using collaborative filtering with the ListenBrainz database. The goal is to create a model that can recommend artists to users based on their listening history, by identifying patterns in the listening behavior of similar users.

The project consists of two main parts divided in two different Colab Python Notebooks: data processing and model generation.

In the data processing step, the files containing the listening history of the users from the ListenBrainz database are parsed and transformed into a final csv file containing the id of each user, an id for each artist and the playcount of each artist by that user. 

This file is then used in the model generation step, where a sparse matrix is created from the user and artist indexes, and the nearest neighbor's similarity is computed using alternating least squares. The model's accuracy is evaluated by testing it on a set of artists and comparing its recommendations to those of other recommender systems.

Technologies used in this project include Python, Pandas, NumPy, and the implicit library for collaborative filtering.

## Data
---
The ListenBrainz data is distributed in 12 files, one for each month of 2022.

Every listen is identified by a `recording_msid` that is random (the same song could have multiple different `recording_msid`). To compare the data we need to convert this id into a `recording_mbid` that is present in the database.

To do this there are three additional files:
- A mapping between each `recording_msid` and `recording_mbid` in the  ListenBrainz database (`listenbrainz_msid_mapping.csv`)
- The ListenBrainz Canonical Metadata database, which includes a list of all `artist_mbid` (`canonical_musicbrainz_data.csv`).
- `artist_mbid` to artist name mapping, with the name of all artists in the MusicBrainz database. (`musicbrainz_artist_mbid_name.csv`)

In [1]:
# I'm working on Google Drive so I mount my Drive into Colab and change the working directory to the project folder.
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/AMPLab/LargeScaleDatasets/CollaborativeFiltering'
os.chdir(path)

Mounted at /content/drive


In [2]:
# I'm using the jsonlines and csv libraries as well as the tqdm to track the progress of the data processing.
!pip install jsonlines
import jsonlines, csv
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# The path of each of the folders that I'm going to use during the project is saved into a variable.
listensFolder = 'ListenBrainzData/listenbrainz-2022'
mappingsFolder = 'ListenBrainzData/'
metaBrainzFolder = 'ListenBrainzData/metabrainz-metadata-dump-20230117-172210/metabrainz'

In [8]:
# This is what the files of the user's listen history looks like
!head ListenBrainzData/listenbrainz-2022/1.listens

{"user_id":17240,"user_name":"Winterbay","timestamp":1640995200,"track_metadata":{"artist_name":"Tiken Jah Fakoly","release_name":"Coup De Gueule","additional_info":{"artist_msid":"f03fa31b-a428-47e2-8dcb-bceec9ca1221","release_msid":"238bd863-a293-4718-a66b-093ea54bf8f3","listening_from":"lastfm","recording_msid":"f216e2fb-784d-470a-81b2-6e27cd532204","lastfm_artist_mbid":"edef3cfa-4e5e-4d64-8bd8-20f9dc1d8cad","lastfm_release_mbid":"9dc7fe6a-3fa4-4461-8975-ecb7218b39a3"},"track_name":"Alou Maye"},"recording_msid":"f216e2fb-784d-470a-81b2-6e27cd532204"}
{"user_id":16930,"user_name":"kazoo","timestamp":1640995200,"track_metadata":{"artist_name":"Fraktus","release_name":"Millennium Edition","additional_info":{"artist_msid":"afe0b08d-f47d-4adf-bd78-3e95ca276f7c","tracknumber":11,"release_msid":"9cd2285c-4860-4e83-b8bb-5fef7328b224","recording_msid":"7a490913-f4d5-40a8-b34a-02166fbc511e"},"track_name":"Computerliebe"},"recording_msid":"7a490913-f4d5-40a8-b34a-02166fbc511e"}
{"user_id":1513

To compute the nearest neighbor's similarity between the different users we need to reduce all the given data to `user_id`, `artist_id`, `artist_name` and `playcount`. All the data processing is going to aim to get a .tsv file with this data.

In [6]:
# We ara analyzing 12 files containing the users' listening history of each month of 2022. Each of these files' size is around 3GB so loading all that data
# to memory is not an option. Because of that we take from the listens file just the needed information: user_id that characterize the user and a recording_msid
# that uniquely identifies a recording or a song. This data is saved into a new .csv file

overwrite = False

if overwrite:
    with open('userID_to_msid.csv', 'w') as writer:
        line2write = 'user_id,recording_msid\n'
        writer.write(line2write)
        for i in range(12):
            listensFile = f'{i+1}.listens'
            print(f'Processing file {listensFile}')
            with jsonlines.open(os.path.join(listensFolder, listensFile)) as reader:
                for line in reader:
                  line2write = f'{line["user_id"]},{line["recording_msid"]}\n'
                  writer.write(line2write)
    print('userID_to_msid.csv file generated.')

else:
    print('File not overwritten.')

Processing file 1.listens
Processing file 2.listens
Processing file 3.listens
Processing file 4.listens
Processing file 5.listens
Processing file 6.listens
Processing file 7.listens
Processing file 8.listens
Processing file 9.listens
Processing file 10.listens
Processing file 11.listens
Processing file 12.listens
userID_to_msid.csv file generated.


In [ ]:
# This is what the new .csv file looks like

!head userID_to_msid.csv

user_id,recording_msid
17240,f216e2fb-784d-470a-81b2-6e27cd532204
16930,7a490913-f4d5-40a8-b34a-02166fbc511e
15136,bc94c2a8-f3e3-4539-b2e9-a775a431cdd2
1626,30f3f6b6-3d76-48e9-b802-dcd5200958f3
15276,f0890cbf-2aa1-44dd-b3d9-ba385f2f4855
1139,51bcd76a-c452-425e-a804-cf59b78cd71e
249,1160f670-c889-4f09-9e34-e4e22d5717d8
1139,6a4fb4ca-52eb-4962-a9a6-0f3b22fb265c
11257,f603f6ef-e476-4dda-8ad5-54835b1c074e


In [ ]:
# This is the number of lines of the .csv file

!cat userID_to_msid.csv | wc -l

48273772


In [ ]:
# From the userID_to_msid.csv file we are going to get all the unique recording_msid to get the corresponding recording_mbid for these ones.

recording_msid = set()

with open('userID_to_msid.csv', 'r') as f:
    reader = csv.DictReader(f)
    for line in reader:
        recording_msid.add(line['recording_msid'])

In [ ]:
# This is what the mapping file looks like.

!head ListenBrainzData/listenbrainz_msid_mapping.csv

recording_msid,recording_mbid,match_type
00000737-3a59-4499-b30a-31fe2464555d,1fe669c9-5a2b-4dcb-9e95-77480d1e732e,exact_match
000013b3-dbb4-43a0-8fd4-ca92ff5ed033,c5bfd98d-ccde-4cf3-8abb-63fad1b6065a,exact_match
00002714-6f74-409d-9fa4-441c8dfb195f,c6acc112-3df7-4716-b5b6-953b5e93743f,exact_match
00003a81-2a6c-4d6c-ad43-990c0806458b,007770e2-90c5-49f2-b894-690db7ebea40,exact_match
00005660-7eb0-4592-a74b-14f3de9cc4cb,67bcde07-bfb1-4b30-88ba-6b995ec04123,exact_match
00006a3b-babd-4bb0-89b3-e2835aba6425,7d52ec8a-3d84-4e3d-8463-657d749ba4d4,exact_match
0000825d-b547-43a7-b294-948e8e472766,99687951-9b5d-4918-b08e-39921d736d68,high_quality
00008dc7-e09c-451e-85a0-7ce1db8fbe19,e44e1b73-9750-4712-a1c3-f57ae2a84f5a,med_quality
00009fc5-4f28-4020-b02c-966d6c5e4202,1dd0bc8f-f8c3-4658-b397-ec9ec87e618f,exact_match


In [ ]:
# From the set of the unique recording_msid's and the mapping file we make a recording_msid to recording_mbid dictionary and save it to a .csv
# file so in case something goes wrong I don't need to repeat the processing.

subset_msid_mbid = {}

mappingFile = os.path.join(mappingsFolder, 'listenbrainz_msid_mapping.csv')

with open(mappingFile, 'r') as fr, open('subset_msid_mbid.csv', 'w') as writer:
    line2write = 'recording_msid,recording_mbid\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        if line['recording_msid'] in recording_msid:
            subset_msid_mbid[line['recording_msid']] = line['recording_mbid']
        if reader.line_num % 1_000_000 == 0:
            for key, value in subset_msid_mbid.items():
                line2write = f'{key},{value}\n'
                writer.write(line2write)
            subset_msid_mbid = {}
    for key, value in subset_msid_mbid.items():
        line2write = f'{key},{value}\n'
        writer.write(line2write)
    subset_msid_mbid = {}
    del recording_msid

In [ ]:
# This is the length in lines of the dictionary .csv

!cat subset_msid_mbid.csv | wc -l

11291748


In [ ]:
# This is what the dictionary .csv file looks like

!head subset_msid_mbid.csv

recording_msid,recording_mbid
000013b3-dbb4-43a0-8fd4-ca92ff5ed033,c5bfd98d-ccde-4cf3-8abb-63fad1b6065a
0000c7ce-5855-4259-823f-fd2e1e4615ce,22e1f70a-df0a-4de3-aa65-2694b7308b2b
00019a12-08fa-46dc-bf78-b1dbfb5b9ae2,a3d61638-2eb4-4833-8f07-47578247a480
00021542-d0f2-4331-bb9e-aba42f350d8e,86c0470c-f497-43b5-b790-7fe6e46e5e73
0002dd17-b9ec-497d-89be-837c25b6dc82,96f4933a-d82d-4219-ab38-b038ee9ce538
0005a74c-2c36-4951-a771-bc2b8ca903ba,9b94fe85-0b72-43e3-94d1-bcb9ea25212d
000631fd-9e97-4c5a-9d71-c9bd579c6c3b,9fd7ad7f-5f03-40eb-a4c7-126c4e3914ae
000658a9-ec8f-4ead-baeb-4580ed29799b,9fb06ce0-6220-49a3-be50-c609ff7ac143
000682e9-e235-4168-9bbf-cfd015c17116,83125987-7b99-4934-89b7-234eec16eb19


In [10]:
# Reading the msid to mbid dictionary from the .csv file

msid_mbid_dict = {}

with open('subset_msid_mbid.csv', 'r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        msid_mbid_dict[line['recording_msid']] = line['recording_mbid']

In [11]:
# Now we go back and replace the msid with the mbid in the user_id to msid mapping. There might be some msid without a match in the database
# so we need to control the possible KeyError and track how many of this songs without a match happen in our data.

error_cnt = 0

with open('userID_to_msid.csv', 'r') as fr, open('userID_to_mbid.csv', 'w') as writer:
    line2write = 'user_id,recording_mbid\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        try:
          line2write = f'{line["user_id"]},{msid_mbid_dict[line["recording_msid"]]}\n'
          writer.write(line2write)
        except KeyError:
          error_cnt += 1

In [12]:
error_cnt

0

In [ ]:
# The new .csv file looks like

!head userID_to_mbid.csv

user_id,recording_mbid
17240,89346a2d-a595-4afe-a4c8-722fc6f93c61
16930,755014b7-f235-4172-991d-f2eaf8450e9d
15136,ae4f8491-0828-4857-936a-ccc1e87f6573
1626,8a7e3912-9df2-49ba-9ed0-1a4fb5dd9ba4
15276,6c48fb39-691d-4959-b1e9-a830df2f1090
1139,1fc2055b-2493-44a8-9ee2-c5cf17b00b17
249,edda2b10-c034-42ed-be7f-d9ccf4254fe1
1139,4670a763-cf40-4921-bd7d-3d92616d76dd
11257,


In [ ]:
# I repeat the process with the recording_mbid. First, I create a set containing the unique r_mbid contained in the data.

r_mbid = set()

with open('userID_to_mbid.csv', 'r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        r_mbid.add(line['recording_mbid'])

In [ ]:
# This is what the canonical_musicbrainz_data.csv file looks like

!head ListenBrainzData/metabrainz-metadata-dump-20230117-172210/metabrainz/canonical_musicbrainz_data.csv

id,artist_credit_id,artist_mbids,artist_credit_name,release_mbid,release_name,recording_mbid,recording_name,combined_lookup,score,year
28939355,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,5f5f649f-1938-4a3e-a879-a95693a99a71,Heavenly Flow,michieoneheavenlyflow,371181,2006
28939356,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,6ee381af-e9b4-46c4-a4f0-541dce2f03ea,Party,michieoneparty,371181,2006
28939357,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,8b371ea0-dee1-4fbf-bacd-3aa87a4aef13,Free Like Jah,michieonefreelikejah,371181,2006
28939358,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-9ab6-ad50067303ee,Power of One,9d904f0f-314b-4089-b151-d81e0857c431,People,michieonepeople,371181,2006
28939359,1415161,{5e3071a8-8c56-4ab2-91f6-c76d35388dbd},Michie One,430bd180-0f13-4144-

In [ ]:
# Once again we make a dictionary of the subset of recording_mbid to artist_mbid that appears in the data and save it to a .csv file.

subset_mbid_artist = {}
mappingFile = os.path.join(metaBrainzFolder, 'canonical_musicbrainz_data.csv')

with open(mappingFile, 'r') as fr, open('subset_mbid_artist.csv', 'w') as writer:
    line2write = 'recording_mbid,artist_mbids\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        if line['recording_mbid'] in r_mbid:
            subset_mbid_artist[line['recording_mbid']] = line['artist_mbids']
        if reader.line_num % 1_000_000 == 0:
            for key, value in subset_mbid_artist.items():
                line2write = f'{key},{value}\n'
                writer.write(line2write)
            subset_mbid_artist = {}
    for key, value in subset_mbid_artist.items():
        line2write = f'{key},{value}\n'
        writer.write(line2write)
    subset_mbid_artist = {}
    del r_mbid

In [ ]:
!cat mbid_artist_subset.csv | wc -l

2881609


In [ ]:
# This is what the dictionary csv file looks like.

!head mbid_artist_subset.csv

recording_mbid,artist_mbids
17d1823d-e43f-4f35-8640-af779503e298,{01cbcfb3-cd44-4e84-815e-a971e23ce38d}
5a19e85f-06fb-41e3-9b98-c0a6d1b2215c,{01cbcfb3-cd44-4e84-815e-a971e23ce38d}
48fa77df-6486-4dda-b4a9-7f93d8ff22e6,{c9c12ef8-e49e-420c-8e45-d8ea12e383d3}
74788b71-524b-494e-aa03-15aae3c4825c,{c9c12ef8-e49e-420c-8e45-d8ea12e383d3}
b7f2540f-571f-4b5c-a55a-d43ff0ba7a99,{c9c12ef8-e49e-420c-8e45-d8ea12e383d3}
c245bad3-1e7f-4afe-b468-f114e866ab97,{c9c12ef8-e49e-420c-8e45-d8ea12e383d3}
7c751662-9cf1-4078-8402-cc4ccc7d9aad,{18cd0aa1-91ab-48be-b35d-e38e66145b0d}
63d8f47a-90cf-45b5-9220-dcea50c83267,{b6bb04b5-0f72-42b6-8dc4-257e276e885e}
002de90d-8638-4714-a393-090c84b25172,{1eeb1434-f930-4025-bbc7-1efdcd986654}


In [ ]:
# This just loads the recording_mbid to artist_mbid dictionary from the .csv file.

mbid_artist_dict = {}

with open('mbid_artist_subset.csv', 'r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        mbid_artist_dict[line['recording_mbid']] = line['artist_mbids']

In [ ]:
# Now we go back and replace the recording_mbid with the artist_mbid in the user_id to mbid mapping.

error_cnt = 0

with open('userID_to_mbid.csv', 'r') as fr, open('userID_to_artist.csv', 'w') as writer:
    line2write = 'user_id,artist_mbids\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        try:
          line2write = f'{line["user_id"]},{mbid_artist_dict[line["recording_mbid"]]}\n'
          writer.write(line2write)
        except KeyError:
          error_cnt += 1

In [ ]:
# This is the amount of recording_mbid that don't have a matching artist.

error_cnt

12791697

In [ ]:
!cat userID_to_artist.csv | wc -l

35482075


In [ ]:
# This is what the new .csv file looks like

!head userID_to_artist.csv

user_id,artist_mbids
16930,{4341463c-3bc5-483a-bcff-d32e4650c9b6}
15136,{85a55a8c-c042-40d6-9dc5-fb85d0d926cf}
1626,{77c167d2-4965-4421-830a-9815e4956475}
15276,{44223a23-a89a-4815-9eab-e36ce9d0ebae}
1139,{4ce9435f-03d8-4b46-9d3b-5d830f97562b}
249,{be465d4f-c28d-4ba1-94ab-ebaada7db8af}
1139,{4ce9435f-03d8-4b46-9d3b-5d830f97562b}
16705,{1bfa27e3-0376-4206-a772-4586e25a64f5}
12647,{9968756b-d504-4c90-a184-776ba1dd32ea}


In [ ]:
# From that file we create a new set of the unique artist_mbid's that appear in the data to create a dictionary of artist_mbid to the artist's name.

unique_artistmbid = set()

with open('userID_to_artist.csv','r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        unique_artistmbid.add(line['artist_mbids'].replace('{','').replace('}',''))

In [ ]:
# To do this we use the provided mbid to name mapping.

!head ListenBrainzData/musicbrainz_artist_mbid_name.csv

mbid,name
fadeb38c-833f-40bc-9d8c-a6383b38b1be,Доктор Сатана
49add228-eac5-4de8-836c-d75cde7369c3,Pete Moutso
c112a400-af49-4665-8bba-741531d962a1,Zachary
ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b,The Silhouettes
7b4a548e-a01a-49b7-82e7-b49efeb9732c,Aric Leavitt
60aca66f-e91a-4cb5-9308-b6e293cd833e,Fonograff
3e1bd546-d2a7-49cb-b38d-d70904a1d719,Al Street
df120895-f6c6-4a66-b9cf-73350f0beb61,Love .45
c14f8d3f-ee81-416f-800f-8eff7e77a2e1,Sintellect


In [ ]:
# Now we can make the dictionary of the useful artist_mbid to name and save it to a .csv file.

mappingFile = os.path.join(mappingsFolder, 'musicbrainz_artist_mbid_name.csv')
subset_artistmbid_name = {}

with open(mappingFile, 'r') as fr, open('subset_artistmbid_name.csv', 'w') as writer:
    line2write = 'mbid,name\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        if line['mbid'] in unique_artistmbid:
            subset_artistmbid_name[line['mbid']] = line['name']
        if reader.line_num % 1_000_000 == 0:
            for key, value in subset_artistmbid_name.items():
                line2write = f'{key},{value}\n'
                writer.write(line2write)
            subset_artistmbid_name = {}
    for key, value in subset_artistmbid_name.items():
        line2write = f'{key},{value}\n'
        writer.write(line2write)
    subset_artistmbid_name = {}
    del unique_artistmbid

In [ ]:
# This is the amount of unique artists that appear in the data.
!cat subset_artistmbid_name.csv | wc -l

266521


In [ ]:
# This loads the dictionary from the .csv file again.

artistmbid_name_dict = {}

mappingFile = os.path.join(mappingsFolder, 'musicbrainz_artist_mbid_name.csv')

with open(mappingFile, 'r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        artistmbid_name_dict[line['mbid']] = line['name']

In [ ]:
# Now for every user, we count the amount of times they listened to each artist and make a dictionary of {(user, artist): count}

userIDArtist_to_Count = {}

with open('userID_to_artist.csv','r') as fr:
    reader = csv.DictReader(fr)
    for line in reader:
        if (line['user_id'], line['artist_mbids']) not in userIDArtist_to_Count.keys():
            userIDArtist_to_Count[line['user_id'], line['artist_mbids']] = 1
        else:
            userIDArtist_to_Count[line['user_id'], line['artist_mbids']] = userIDArtist_to_Count[line['user_id'], line['artist_mbids']] + 1

In [ ]:
# The dictionary is saved into a .csv file. Some formatting was needed.

with open('userID_artist_count.csv', 'w') as writer:
    line2write = 'user_id,artist_mbids,playcount\n'
    writer.write(line2write)
    for key,value in userIDArtist_to_Count.items():
        line2write = f'{key},{value}\n'.replace(' ','').replace('(','').replace(')','').replace('{','').replace('}','')
        writer.write(line2write)

In [ ]:
# This is what the user_id, artist, playcount .csv file looks like.
!head userID_artist_count.csv

user_id,artist_mbids,playcount
'16930','4341463c-3bc5-483a-bcff-d32e4650c9b6',3
'15136','85a55a8c-c042-40d6-9dc5-fb85d0d926cf',41
'1626','77c167d2-4965-4421-830a-9815e4956475',234
'15276','44223a23-a89a-4815-9eab-e36ce9d0ebae',3
'1139','4ce9435f-03d8-4b46-9d3b-5d830f97562b',5
'249','be465d4f-c28d-4ba1-94ab-ebaada7db8af',3
'16705','1bfa27e3-0376-4206-a772-4586e25a64f5',247
'12647','9968756b-d504-4c90-a184-776ba1dd32ea',9
'19131','d5cc67b8-1cc4-453b-96e8-44487acdebea',87


In [ ]:
# We can now use the artist_mbid to name dictionary to decode the artist names 
artistmbid_name_dict['4341463c-3bc5-483a-bcff-d32e4650c9b6']

'Fraktus'

In [ ]:
# Finally we can put everything together into a .tsv file containing the userID, the artist_mbid, the artist name and the playcount.
error_cnt = 0

with open('userID_artist_count.csv', 'r') as fr, open('userID_artistmbid_artistName_plays.tsv', 'w') as writer:
    line2write = 'user_id\tartist_mbids\tartist_name\tplaycount\n'
    writer.write(line2write)
    reader = csv.DictReader(fr)
    for line in reader:
        try:
            line2write = str(line['user_id']).replace('\'','') + '\t' + str(line['artist_mbids']).replace('\'','') +'\t' + str(artistmbid_name_dict[line['artist_mbids'].replace('\'','')]) + '\t' + str(line['playcount']) + '\n'
            writer.write(line2write)
        except KeyError:
            error_cnt += 1

In [ ]:
error_cnt

60

In [ ]:
!cat userID_artistmbid_artistName_plays.tsv | wc -l

3655006


In [ ]:
# This is what the processed data looks like ready to be processed to generate the nearest neighbor's recommender system.

!head userID_artistmbid_artistName_plays.tsv

user_id	artist_mbids	artist_name	playcount
16930	4341463c-3bc5-483a-bcff-d32e4650c9b6	Fraktus	3
15136	85a55a8c-c042-40d6-9dc5-fb85d0d926cf	Népal	41
1626	77c167d2-4965-4421-830a-9815e4956475	Dinosaur Jr.	234
15276	44223a23-a89a-4815-9eab-e36ce9d0ebae	Rappin’ 4‐Tay	3
1139	4ce9435f-03d8-4b46-9d3b-5d830f97562b	Mord'A'Stigmata	5
249	be465d4f-c28d-4ba1-94ab-ebaada7db8af	IDLES	3
16705	1bfa27e3-0376-4206-a772-4586e25a64f5	Milton Nascimento	247
12647	9968756b-d504-4c90-a184-776ba1dd32ea	Sugar Ray	9
19131	d5cc67b8-1cc4-453b-96e8-44487acdebea	Beach House	87
